<a href="https://colab.research.google.com/github/hank199599/data_science_from_scratch_reading_log/blob/main/Chapter20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 集群 (Clusering) → 非監督式學習(unsupervised learning)
沒有哪一種區分方式是一定正確的，  
區分方式端看是否符合區分者的**需求**而定。

# 模型
每個都是d維空間的一個向量，  
世個資料點分配到人個集群中,方法非常 之多,這也就表示要找出最佳化的集 是一個非常国離的問題,我們經常會採用一種送代演算法,它通常可以得出相然不 群是 :

1. 一開始先從維空即中選出k個資料點,做為集群的大平均值(k-means).
2. 把其餘的部個資料點,分配給距離最近的平均值。
3. 如果資科點的分配結集已裡不再變動,就停止下來,並以目前分配的集非故乡作奖 4.如果分配结果仍在變動,就重新計算平均值,然後回到步。2.

只要運用「第4章:線性代數」的 Vector_mean 函式,建立一個物件類別來進行這項工作 整件事就變得提孤單了,

In [1]:
from typing import List

Vector = List[float]

def num_differences(v1:Vector,v2:Vector)->int:
  assert len(v1) == len(v2)
  return len([x1 for x1,x2 in zip(v1,v2) if x1 != x2])
  

In [2]:
assert num_differences([1,2,3],[2,1,3]) == 2

In [3]:
assert num_differences([1,2],[1,2]) == 0

### 建立函式：  
* 給定一堆向量以及各向量所對應的集群 → 計算各集群的平均值

In [5]:
from typing import List

def vector_mean(vectors:List[Vector])->Vector:
  n=len(vectors)
  return scalar_multiply(1/n,vector_sum(vectors))

def cluster_means(k:int,inputs:List[Vector],assignment:List[int]) ->List[Vector]:

  # clusters[i] 包含的是分配給集群i的輸入資料點
  cluster = [[] for i in range(k)]

  for input,assigment in zip(inputs,assigments):
    clusters[assignment].append(input)
  
  # 如果集群是空的，就用一個隨機點來做為其代表
  return [vector_mean(cluster) if cluster else random.choice(inputs) for cluster in clusters]

### 集群處理的程式碼

In [6]:
import itertools
import random
import tqdm

def squared_distance(v:Vector,w:Vector) -> float:
  return sum_of_squares(subtrate(v,w))

In [ ]:
class KMeans:
  def __init__(self,k:int)->None:
    self.k = k
    self.means = None
  
  def classify(self,inputs:List[Vector])->None:
    """送回最接近輸入的集群索引值"""
    return min(range(self.k),key=lambda i:squared_distance(input,self.means[i]))
  
  def train(self,inputs:List[Vector])->None:
    for _ in t:
      # 計算平均值並找出新的集群分配結果
      self.means = cluster_means(self.k,inputs,assignment)
      new_assigments = [self.classify(input) for input in inputs]

      # 檢查有幾個分配結果出現變化，並判斷是否已經完成
      num_changed = num_differences(assignment,new_assigments)
      if num_changed == 0:
        return
      
      # 如果尚未完成，就繼續重新分配集群，並計算新的平均值
      assignment = new_